In [1]:
import torch
from test_sparse import *
from utils import *
import json

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ValueError: Customized func is already registered in autoTVM task dynconv/pooling

In [4]:
rr = []
ss = [0.2, 0.4, 0.6, 0.8]
sd = 0.01
widths, channels, group = regnet_parameters("008")
for i in range(4):
    width, channel = widths[i], channels[i]
    gg = get_factors(width)
    for s in ss:
        for g in gg:
            bs = TVMDynamicBlockEvaluator(channel, width, group, f"log/c{channel}_w{width}_g{group}")
            sl = find_best_sparselen(width//g, s-sd, s+sd)
            bs.setup_sparse(sl, g, True, True, True, True)
            bs.build("sparse")
            res = bs.evaluate("sparse", verbose=True)
            res["channel"] = channel
            res["width"] = width
            res["group"] = group
            res["sparsity"] = s
            res["sparselen"] = sl
            res["granularity"] = g
            rr.append(res)
log_ls_to_json(rr, "results.json")


In [5]:
import pandas as pd

# rr = load_json_to_ls("results.json")
df = pd.DataFrame(columns=list(rr[0].keys()))
for r in rr:
    r_ = {k: v for k, v in r.items()}  
    for m in ["conv2", "conv3", "maskconv", "pooling", "gather", "scatter_add"]:
        r_[m] *= 1000 # in ms
    df = df.append(r_, ignore_index=True)
print(df)
df.to_csv("results.csv")

/tmp/ipykernel_20296/496098352.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(r_, ignore_index=True)
/tmp/ipykernel_20296/496098352.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(r_, ignore_index=True)
/tmp/ipykernel_20296/496098352.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(r_, ignore_index=True)
/tmp/ipykernel_20296/496098352.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(r_, ignore_index=True)
/tmp/ipykernel_20296/496098352.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat

       conv2     conv3    gather scatter_add  maskconv   pooling channel  \
0   0.004855  0.005187  0.005641    0.003446   0.00393  0.003098    64.0   
1   0.004291  0.004662  0.003806    0.003317  0.003269  0.003093    64.0   
2   0.003741  0.003952  0.003349    0.003342  0.003193  0.003059    64.0   
3   0.003897   0.00511  0.003546    0.003301  0.003005  0.003033    64.0   
4    0.00413  0.004775  0.002952    0.003308  0.003022  0.003543    64.0   
..       ...       ...       ...         ...       ...       ...     ...   
59  0.010315  0.013157  0.003336    0.002858  0.004893  0.002766   288.0   
60   0.01232  0.012326  0.003051    0.002785  0.009693  0.002723   672.0   
61  0.013941  0.017109  0.003346    0.002918  0.009693  0.002726   672.0   
62  0.019561  0.020927  0.003577    0.002841  0.009698  0.002739   672.0   
63  0.023167   0.02417  0.004082     0.00327  0.009686  0.002747   672.0   

   width group sparsity sparselen granularity  
0   56.0  16.0      0.2     600.0      

/tmp/ipykernel_20296/496098352.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(r_, ignore_index=True)
/tmp/ipykernel_20296/496098352.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(r_, ignore_index=True)
/tmp/ipykernel_20296/496098352.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(r_, ignore_index=True)
/tmp/ipykernel_20296/496098352.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(r_, ignore_index=True)
/tmp/ipykernel_20296/496098352.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat